# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921681489570                   -0.69    5.8         
  2   -7.926163998720       -2.35       -1.22    1.0    253ms
  3   -7.926837670084       -3.17       -2.37    1.9    339ms
  4   -7.926861522304       -4.62       -3.04    2.8    300ms
  5   -7.926861644593       -6.91       -3.41    1.8    253ms
  6   -7.926861670030       -7.59       -3.80    1.6    292ms
  7   -7.926861680396       -7.98       -4.33    1.2    242ms
  8   -7.926861681757       -8.87       -4.83    2.1    267ms
  9   -7.926861681861       -9.98       -5.29    1.9    281ms
 10   -7.926861681871      -11.01       -5.83    2.0    273ms
 11   -7.926861681873      -11.86       -6.77    1.8    260ms
 12   -7.926861681873      -13.23       -7.61    2.6    295ms
 13   -7.926861681873      -15.05       -7.92    3.5    326ms
 14   -7.926861681873   +    -Inf       -9.18    1.2    244ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921723076679                   -0.69    5.6         
  2   -7.926171107757       -2.35       -1.22    1.0    258ms
  3   -7.926837167528       -3.18       -2.37    1.6    331ms
  4   -7.926861494363       -4.61       -3.02    2.9    303ms
  5   -7.926861643582       -6.83       -3.38    1.9    267ms
  6   -7.926861669164       -7.59       -3.77    1.6    257ms
  7   -7.926861680324       -7.95       -4.31    1.2    283ms
  8   -7.926861681796       -8.83       -4.92    2.2    276ms
  9   -7.926861681853      -10.25       -5.16    2.0    269ms
 10   -7.926861681871      -10.75       -5.81    1.6    271ms
 11   -7.926861681873      -11.72       -6.70    2.0    273ms
 12   -7.926861681873      -13.36       -6.96    2.8    296ms
 13   -7.926861681873   +  -15.05       -7.90    1.1    254ms
 14   -7.926861681873      -14.75       -8.03    3.2    309ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921682897066                   -0.69    5.9         
  2   -7.926166211434       -2.35       -1.22    1.0    260ms
  3   -7.926840246595       -3.17       -2.37    1.8    294ms
  4   -7.926864922926       -4.61       -3.01    2.8    369ms
  5   -7.926865046229       -6.91       -3.35    1.8    264ms
  6   -7.926865078459       -7.49       -3.74    1.9    274ms
  7   -7.926865091713       -7.88       -4.37    1.0    240ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       
